In [ ]:
%pip install pymysql

In [ ]:
import pymysql

In [ ]:
conexao = pymysql.connect(
host="dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com",
port=3306,
user="candidate_user",
password="D3@bGh664%$1VHv*")

In [ ]:
exec = conexao.cursor()
query = """
describe db_hiring_test.raw_transactions_table;
"""
exec.execute(query)
desc = exec.fetchall()
print(desc)


In [ ]:
# 1. Qual é o endereço (carteira) com maior volume de transações enviadas?
exec = conexao.cursor()
query = """
select AddressOrigin, 
count(AddressOrigin) qtd_transacoes 
from db_hiring_test.raw_transactions_table 
where Status = "Confirmed"
group by AddressOrigin 
order by count(AddressOrigin) desc 
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Maior numero de transções")
print(desc)

exec = conexao.cursor()
query = """
select AddressOrigin, 
sum(replace(TotalSent, ',', '')) total  
from db_hiring_test.raw_transactions_table 
where Status = "Confirmed"
group by AddressOrigin 
order by sum(replace(TotalSent, ',', '')) desc 
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Maior valor total")
print(desc)

In [ ]:
# 2. Qual é o dia do mês com maior volume de transações realizadas?
exec = conexao.cursor()
query = """
select month(SentDate), 
count(month(SentDate)) qtd_transacoes 
from db_hiring_test.raw_transactions_table 
group by month(SentDate) 
order by count(month(SentDate)) desc 
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Maior numero de transções por mês")
print(desc)

exec = conexao.cursor()
query = """
select month(SentDate), 
sum(replace(TotalSent, ',', '')) total  
from db_hiring_test.raw_transactions_table 
group by month(SentDate) 
order by sum(replace(TotalSent, ',', '')) desc 
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Maior valor total por mês")
print(desc)

In [ ]:
# 3. Em qual dia da semana geralmente mais transações são realizadas?
exec = conexao.cursor()
query = """
select 
case 
    when weekday(SentDate) = 0 then "Segunda"
    when weekday(SentDate) = 1 then "Terça"
    when weekday(SentDate) = 2 then "Quarta"
    when weekday(SentDate) = 3 then "Quinta"
    when weekday(SentDate) = 4 then "Sexta"
    when weekday(SentDate) = 5 then "Sabado"
    else "Domingo"
END, 
count(weekday(SentDate)) 
from db_hiring_test.raw_transactions_table 
group by weekday(SentDate) 
order by count(weekday(SentDate)) desc 
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Maior numero de transções por dia da semana")
print(desc)

In [ ]:
# 4. Quais transações possuem condições atípicas e precisam ser validadas com o time 
# responsável pela disponibilização dos dados?
exec = conexao.cursor()
query = """
select *
from db_hiring_test.raw_transactions_table 
where Status = 'Denied';
"""
exec.execute(query)
desc = exec.fetchall()
print("Não entendi muito bem o que seria condições atípicas, mas creio que seja isso:")
print(desc)

In [ ]:
# 5. Qual a carteira com o maior saldo final? (considere que todas as carteiras estejam zeradas no 
# início das análises e que seja possível existir saldo negativo).

exec = conexao.cursor()
query = """select orig.AddressOrigin, (dest.destino - orig.origen) total  from (
select AddressOrigin, sum(TotalSent) origen 
from db_hiring_test.raw_transactions_table group by AddressOrigin
) orig 
left join (
select AddressDestination, sum(TotalSent) destino
from db_hiring_test.raw_transactions_table group by AddressDestination
) dest 
on AddressOrigin = AddressDestination
order by (dest.destino - orig.origen) desc
limit 1;
"""
exec.execute(query)
desc = exec.fetchall()
print("Carteira com maior saldo:")
print(desc)